In [34]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from collections import Counter

In [35]:
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")
N = train.shape[0]

In [36]:
# def category(i):
#     a = i / 26
#     b = i % 26
#     return chr(ord('a') + a) + chr(ord('a') + b)

In [37]:
frac = 0.1 # fraction of learning examples used for model fitting

# sample the train set if your don't want to deel with all examples
#train = train.tail(n=int(N * frac))
train = train.sample(frac=frac, random_state=42)

N = train.shape[0]

train['week'] = train['week'] - train['shift']

for i in range(N):
    if train['week'].values[i] <= 0:
        train['week'].values[i] = (train['week'].values[i] + 52) % 53
        train['year'].values[i] = (train['year'].values[i] - 1)

train = train.drop(['shift'], axis=1)

train = train.sort(['year', 'week'])

# item_id = np.array(train['item_id'])
# counter = Counter(item_id)
# for i, x in enumerate(counter.items()):
#     counter[x[0]] = category(i)
    
#train['item_id'] = train['item_id'].astype(int).astype('str')

# cat = []
# for i in range(N):
#     cat.append(counter[train['item_id'].values[i]])
# train['item_id'] = train['item_id'].astype('str')
# for i in range(N):
#     train['item_id'].values[i] = cat[i]

X = train.drop(['Num', 'y'], axis=1)

print X

y = train['y'].reshape(N, 1)
y = [elem[0] for elem in y]

       year  week   item_id         f1         f2         f3         f4  \
484    2012    51  20442117     3845.0    29629.0    27762.0    32443.0   
127    2012    51  20438572    20650.0   198032.0   215778.0   260442.0   
274    2012    51  20441125     7545.0    72968.0    70995.0    89973.0   
267    2012    51  20440561     3310.0    44720.0    44590.0    85470.0   
444    2012    51  20438693    11800.0   130975.0   129141.0   173085.0   
564    2012    51  20443307     1981.0    42254.0    23520.0    25145.0   
454    2012    51  20438716    22810.0   294010.0   326995.0   425472.0   
176    2012    51  20438332     8020.0    71982.0    78962.0   112012.0   
524    2012    51  20448467      695.0     7613.0     6895.0     8200.0   
128    2012    51  20438581        0.0        0.0        0.0       10.0   
253    2012    51  20442114      160.0     2385.0     1920.0     2380.0   
446    2012    51  20438702    24690.0   266912.0   278919.0   361529.0   
35     2012    51  204407

/home/nimloth/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel/__main__.py:18: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/nimloth/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel/__main__.py:38: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [38]:
def smape(y, y_pred, **kwargs):
    y = np.array(y)
    y_pred = np.array(y_pred)
    ans = (np.abs(y - y_pred) * 1.0 / (np.abs(y) + np.abs(y_pred))).mean() * 100
    return ans * 2

from sklearn.metrics import make_scorer
scorer = make_scorer(score_func=smape)

In [39]:
tscv = TimeSeriesSplit(n_splits=3)

In [41]:
for max_depth in [14]:
    model = xgb.XGBRegressor(max_depth=max_depth, n_estimators=100,
                             learning_rate=0.03, seed=95)
    accuracy = cross_val_score(model, X, y, scoring=scorer, cv=tscv)
    print accuracy

[ 31.43133143  26.9236279   32.89132604]


In [ ]:
32.28366249  27.14061112  33.09830808